In [1]:
%run ../utils/starter.ipynb

In [2]:
%run ../utils/helper.ipynb

loaded
loaded


In [3]:
def compare_data(a,b,k):
    return a.data[k]==b.data[k]

In [4]:
def add_axes():
    bpy.ops.object.empty_add(type='PLAIN_AXES', align='CURSOR')
    return context.object

def add_bolt_and_nut(screw_length=6,div_count=36):
    
    bpy.ops.preferences.addon_enable(module="add_mesh_BoltFactory")
    
    #to prevent error
    bpy.ops.object.add(type="MESH")
    ob=bpy.context.active_object
    
    bpy.ops.mesh.bolt_add(align='WORLD',
                      location=(0, 0, 0), rotation=(0, 0, 0),
                      change=False, bf_Model_Type='bf_Model_Bolt',
                      bf_Head_Type='bf_Head_Hex', bf_Bit_Type='bf_Bit_None',
                      bf_Nut_Type='bf_Nut_Hex', bf_Shank_Length=0,
                      bf_Shank_Dia=3, bf_Phillips_Bit_Depth=1.14315,
                      bf_Allen_Bit_Depth=1.5,
                      bf_Allen_Bit_Flat_Distance=2.5,
                      bf_Hex_Head_Height=2, bf_Hex_Head_Flat_Distance=5.5,
                      bf_CounterSink_Head_Dia=6.3, bf_Cap_Head_Height=3,
                      bf_Cap_Head_Dia=5.5, bf_Dome_Head_Dia=5.6,
                      bf_Pan_Head_Dia=5.6, bf_Philips_Bit_Dia=1.82,
                      bf_Thread_Length=screw_length, bf_Major_Dia=3, bf_Pitch=0.35,
                      bf_Minor_Dia=2.62111, bf_Crest_Percent=10,
                      bf_Root_Percent=10, bf_Div_Count=div_count,
                      bf_Hex_Nut_Height=2.4, bf_Hex_Nut_Flat_Distance=5.5)
    bolt=context.object
    
    bpy.ops.mesh.bolt_add(align='WORLD',
                      location=(0, 0, 0), rotation=(0, 0, 0),
                      change=False, bf_Model_Type='bf_Model_Nut',
                      bf_Head_Type='bf_Head_Hex', bf_Bit_Type='bf_Bit_None',
                      bf_Nut_Type='bf_Nut_Hex', bf_Shank_Length=0,
                      bf_Shank_Dia=3, bf_Phillips_Bit_Depth=1.14315,
                      bf_Allen_Bit_Depth=1.5,
                      bf_Allen_Bit_Flat_Distance=2.5,
                      bf_Hex_Head_Height=2, bf_Hex_Head_Flat_Distance=5.5,
                      bf_CounterSink_Head_Dia=6.3, bf_Cap_Head_Height=3,
                      bf_Cap_Head_Dia=5.5, bf_Dome_Head_Dia=5.6,
                      bf_Pan_Head_Dia=5.6, bf_Philips_Bit_Dia=1.82,
                      bf_Major_Dia=3, bf_Pitch=0.35,
                      bf_Minor_Dia=2.62111, bf_Crest_Percent=10,
                      bf_Root_Percent=10, bf_Div_Count=div_count,
                      bf_Hex_Nut_Height=2.4, bf_Hex_Nut_Flat_Distance=5.5)
    nut=context.object
    
    axes=add_axes()
    bolt.parent=axes
    nut.parent=axes
    
    assert compare_data(*get_bolt_and_nut(axes),"bf_Pitch") is True
    
    #to prevent error
    for collection in list(ob.users_collection):collection.objects.unlink(ob)
    if ob.users == 0:bpy.data.objects.remove(ob)
    del ob
    
    return axes
    

In [5]:
def screwing(bolt,nut):
    def to_screw(bolt,nut):
        pitch=bolt.data["bf_Pitch"]

        constraint=bolt.constraints.new(type='TRANSFORM')

        constraint.map_from = 'LOCATION'

        constraint.target_space = 'LOCAL'
        constraint.owner_space = 'LOCAL'
        
        if False:
            #buggy:from rotation to location
            constraint.map_from = 'ROTATION'
            constraint.map_to = 'LOCATION'
            constraint.from_min_z_rot = 6.28319
            constraint.from_max_z_rot = 0
            constraint.to_max_z = pitch
            constraint.to_min_z = 0
        else:
            #wrong
            constraint.map_to = 'ROTATION'
            constraint.map_from = 'LOCATION'
            constraint.to_min_z_rot = 6.28319
            constraint.to_max_z_rot = 0
            constraint.from_max_z = pitch
            constraint.from_min_z = 0
            

        constraint.target = nut

        constraint.use_motion_extrapolate = True
    to_screw(bolt,nut)
    to_screw(nut,bolt)

In [6]:
def limiting(bolt,nut):
    length=bolt.data["bf_Thread_Length"]
    length-=nut.data['bf_Hex_Nut_Height']
    length-=bolt.data['bf_Hex_Nut_Height']
    length+=bolt.data['bf_Pitch']*4.8
    def to_limit_location(nut,negative=False):
        nonlocal length
        
        constraint=nut.constraints.new(type="LIMIT_LOCATION")

        constraint.owner_space = 'LOCAL'
        
        constraint.use_min_x = True
        constraint.use_max_x = True
        constraint.use_min_y = True
        constraint.use_max_y = True
        
        constraint.use_min_z = True
        constraint.use_max_z = not negative
        
        constraint.min_x=0
        constraint.max_x=0
        constraint.min_y=0
        constraint.max_y=0
        
        if negative:
            constraint.min_z=-length
            constraint.max_z=0
        else:
            constraint.min_z=0
            constraint.max_z=length
        
        constraint.use_transform_limit = True

        
    def to_limit_rotation(nut):
        constraint=nut.constraints.new(type="LIMIT_ROTATION")

        constraint.owner_space = 'LOCAL'
        
        constraint.use_limit_x = True
        constraint.use_limit_y = True
        constraint.use_limit_z = False
        
        constraint.min_x=0
        constraint.min_y=0
        constraint.min_z=0
        constraint.max_x=0
        constraint.max_y=0
        constraint.max_z=0
        
        
        constraint.use_transform_limit = True

    to_limit_location(nut)
    to_limit_location(bolt,True)
    
    to_limit_rotation(nut)
    to_limit_rotation(bolt)
    

In [7]:
def get_bolt_and_nut(axes):
    bn=[]
    for i in axes.children:
        if i.data is None:continue
        if "Bolt" in i.data.keys():
            bn.append(i)
    # assert len(bn)=2
    bolt,nut=None,None
    for i in bn:
        if i.name=="Nut":nut=i
        if i.name=="Bolt":bolt=i
    return bolt,nut

def add_screw_constrains(axes):
    bolt,nut=get_bolt_and_nut(axes)
    screwing(bolt,nut)
    limiting(bolt,nut)

In [8]:
def new_bolt_and_nut(*args,**kwargs):
    axes=add_bolt_and_nut(*args,**kwargs)

    add_screw_constrains(axes)
    
    return axes

In [9]:
axes=new_bolt_and_nut(screw_length=10,div_count=100)

In [10]:
axes=new_bolt_and_nut(screw_length=10,div_count=10)

AttributeError: 'NoneType' object has no attribute 'data'

AttributeError: 'NoneType' object has no attribute 'data'

In [ ]:
nut,bolt=get_bolt_and_nut(axes)

In [ ]:
nut.data.ust_auto_smoth=True

In [ ]:
context.object.